In [ ]:
from fastai.vision.all import *
import pandas as pd
from pathlib import Path
from sklearn.model_selection import StratifiedKFold

In [ ]:
path = Path('../input/happy-whale-and-dolphin')

In [ ]:
train_df = pd.read_csv(path/'train.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.nunique() #Getting the number of unique values for each column

In [ ]:
train_df.species.value_counts()

In [ ]:
train_df.individual_id.value_counts().reset_index()

In [ ]:
train_individual_count_df = train_df.individual_id.value_counts().reset_index()
train_individual_count_df.columns = ['id', 'count']
train_individual_count_df[train_individual_count_df['count'] == 5]

In [ ]:
skf = StratifiedKFold(n_splits = 5)
#df = pd.DataFrame()
for fold, (train_index ,val_index) in enumerate(skf.split(X = train_df, y = train_df.individual_id)):
    train_df.loc[val_index, "kfold"] = fold
    '''if fold == 0:
        train_df.loc[val_index, "is_valid"] = True
        #df.loc[train_index, "is_valid"] = True
        print(len(train_index))
    else:
        train_df.loc[val_index, "is_valid"] = False
        #df.loc[train_index, "is_valid"] = False
        print(len(train_index))'''
    

In [ ]:
train_df[train_df['individual_id'] == '3509cb6a8504']

In [ ]:
train_df.head()

In [ ]:
train_df['kfold'].value_counts().reset_index()

In [ ]:
#train_df['is_valid'].value_counts()

In [ ]:
train_df.shape

In [ ]:
def splitter(df): 
    train = df.index[df.kfold != 0].tolist()
    valid = df.index[df.kfold == 0].tolist()
    return [train,valid]

In [ ]:
datablock = DataBlock(
    blocks = (ImageBlock, MultiCategoryBlock),
    get_x = ColReader('image', path/'train_images'),
    get_y = ColReader('individual_id'),
    splitter = splitter,
    item_tfms = Resize(256)
)

In [ ]:
dls = datablock.dataloaders(train_df)

In [ ]:
learn = cnn_learner(dls, resnet34, metrics = accuracy_multi)

In [ ]:
learn.fine_tune(5)

In [ ]:
learn.export('happy_whale_model.pkl')

In [ ]:
learn.predict('../input/happy-whale-and-dolphin/test_images/000110707af0ba.jpg')

In [ ]:
sample_img = PILImage.create('../input/happy-whale-and-dolphin/test_images/000110707af0ba.jpg')
sample_img.to_thumb(250)

In [ ]:
sample_submission_df = pd.read_csv(path/'sample_submission.csv')

In [ ]:
sample_submission_df.head(10)

In [ ]:
import os

results = pd.DataFrame(columns = ['image', 'predictions'])
test_dir = path/'test_images'
test_images = os.listdir(test_dir)
for img, iter in enumerate(test_images):
    pred, _, _ = learn.predict(test_dir/img)              
    results.loc[iter] = [img, pred]

In [ ]:
results.head()